<a href="https://colab.research.google.com/github/karolinakuligowska/Projektowanie_systemow_informatycznych/blob/main/PSI_Zaj10_LOT_reviews_csv_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install lxml

In [18]:
!pip install pandas

In [19]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import unicodedata

In [20]:
# --- Settings ---
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36"
}

# --- Scrape only LOT Polish Airlines ---
url = "https://www.airlinequality.com/airline-reviews/lot-polish-airlines/?sortby=post_date%3ADesc&pagesize=100"

html = requests.get(url, headers=headers).text
soup = BeautifulSoup(html, "lxml")

container = soup.find("article", {"class": "comp comp_reviews-airline querylist position-content"})

# --- Initialize ---
columns = ["Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service",
           "Food & Beverages","Ground Service","Inflight Entertainment","Wifi & Connectivity",
           "Value For Money","Recommended"]

df_columns = ["Airline Name", "Overall_Rating", "Review_Title", "Review_Date", "Verified", "Review_Text"] + columns

reviews = []

In [5]:
# https://www.airlinequality.com/airline-reviews/lot-polish-airlines/?sortby=post_date%3ADesc&pagesize=100

In [21]:
# --- Scrape each review ---
if container:
    for items in container.find_all("article"):
        verified = False

        rating_tag = items.find("div", {"class": "rating-10"})
        rating = rating_tag.text.strip()[:1] if rating_tag else None

        title_tag = items.find("h2")
        title = title_tag.text.strip() if title_tag else None

        time_tag = items.find("h3").find("time")
        review_date = time_tag.text.strip() if time_tag else None

        text_block = items.find("div", {"class": "text_content"}).text
        text = text_block.split("|")
        if len(text) == 1:
            review_text = text[0].strip()
        else:
            if text[0].strip() == '✅ Trip Verified':
                verified = True
            review_text = text[1].strip()

        # Now, scrape small table
        table = items.find("table")
        tab = [None]*13  # 13 fields

        if table:
            for row in table.find_all("tr"):
                i = 0
                for td in row.find_all("td"):
                    if i == 0:
                        field_name = td.text.strip()
                        try:
                            index = columns.index(field_name)
                        except:
                            index = None
                        i = 1
                    else:
                        if td.find("span") is None:
                            value = td.text.strip()
                        else:
                            value = len(td.find_all("span", {"class": "star fill"}))
                        if index is not None:
                            tab[index] = value

        # Save data
        data = ["LOT Polish Airlines", rating, title, review_date, verified, review_text] + tab
        reviews.append(data)

        # Pause politely
        time.sleep(2)


In [22]:
# --- Create DataFrame ---
df = pd.DataFrame(reviews, columns=df_columns)

# --- Save to CSV ---
df.to_csv("LOT_reviews.csv", index=False)
